# Зависимости

In [1]:
import csv
import json
from shortcuts import (read_portion_data, append_functions_in_json, 
                       filter_functions_name, dataset_save_to_csv, 
                       load_functions_from_json)
from settings import FUNCTIONS_PATH, FILTERED_FUNCTIONS_PATH, DATASET_PATH

# Выполнение анализа имен функций

### Подсчёт количества функций

In [2]:
with open(FUNCTIONS_PATH, 'r') as json_file:
    func_number = 0
    for line in json_file:
        func_number+=1
print('Общее количество функций в датасете:', func_number)

Общее количество функций в датасете: 1274485


### Подсчет уникальных имен функций и значений узлов

Из-за большого размера файлов оперативная память не справиться с обработкой, поэтому производится порциональный анализ функций.

* `batch_size` - размер порции (1000 для ОЗУ объемом 4 Гб)

* `uniq_names` - словарь уникальных имен функций со стчетчиком

* `uniq_values` - словарь уникальных значений узлов функций

In [3]:
uniq_names = {}
uniq_values = {}

batch_size = 1000
pos = 0 # начальная позиция чтения файла
counter = 0 # счётчик обработанных функций

# Порционный перебор функций из датасета
functions, pos = read_portion_data(FUNCTIONS_PATH, pos, 1)
while functions:
    functions, pos = read_portion_data(FUNCTIONS_PATH, pos, batch_size)
    
    counter += len(functions) 
        
    # Добавить имена в словарь подсчета уникальных имен функций
    for func in functions:
        func = json.loads(func)
        uniq_names[func[0]['value']] = uniq_names.get(func[0]['value'], 0) + 1
        # Добавить имена в словарь подсчета уникальных значений узлов
        for node in func:
            if 'value' in node:
                uniq_values[node['value']] = uniq_values.get(node['value'], 0) + 1
            
    #print('Функций обработано: ', counter, '| Ун. имен', len(uniq_names), '| Ун. знач.:', len(uniq_values))

In [69]:
print(
f'''Функций обработано: {counter}
Уникальных имен функций: {len(uniq_names)}
Уникальных значений узлов: {len(uniq_values)}'''
)
print('Функций потеряно:', func_number-counter)

Функций обработано: 1273210
Уникальных имен функций: 535819
Уникальных значений узлов: 2437048
Функций потеряно: 1275


Импорт/экспорт из/в файл уникальных значений и имен функций.

In [21]:
with open('datasets/uniq_names.json', 'w') as json_file:
    json.dump(uniq_names, json_file)
with open('datasets/uniq_values.json', 'w') as json_file:
    json.dump(uniq_values, json_file)

In [19]:
with open('datasets/uniq_names.json', 'r') as json_file:
    uniq_names = json.load(json_file)
with open('datasets/uniq_values.json', 'r') as json_file:
    uniq_values = json.load(json_file)

### Выбор самых частотных имен функций

* th - (thresholder) задает порог вхождения имен функциий или значений в выборку

In [20]:
name_th = 450
value_th = 25000

In [21]:
def cut_dict(dictionary : dict, thresholder : int) -> set:
    data = []
    for key, count in dictionary.items():
        if count > thresholder:
            data.append(key)
    return set(data)

In [22]:
names = cut_dict(uniq_names, name_th)
values = cut_dict(uniq_values, value_th)

In [23]:
print('Уникальных имен функций: ', len(names))
print('Уникальных значений узлов функций: ', len(values))

Уникальных имен функций:  114
Уникальных значений узлов функций:  166


### Фильтрация имен функций по значению

В выборку попадают функции, которые сложно классифицировать по их сниппету, поэтому такие имена функций исключаются из выборки.

In [24]:
bad_names = ['main', 'run', 'init',
            'iter', 'enter','exit',
            'hash', 'del', 'start',
            'nonzero', 'func', 'foo', '_run']
names = set(filter(lambda name: name not in bad_names and len(name) > 2, names))

In [25]:
print('Уникальных имен функций после чистки на неподходящие имена: ', len(names))

Уникальных имен функций после чистки на неподходящие имена:  101


### Фильтрация уникальных значений

Избавление от значений не несущих много информации

In [26]:
values = set(filter(lambda value: len(value) > 1, values))

In [27]:
print('Уникальных значений после чистки на неподходящие значения: ', len(values))

Уникальных значений после чистки на неподходящие значения:  136


## Фильтрация функций по выбранным именам и трансформация узлов

Все функции из файла `functions.json` будут отфильтрованы по имени и записаны в новый файл `filtered_functions.json`.

### Фильтрация по имени

In [28]:
batch_size = 1000 # Для ОЗУ 8 гигабайт.
pos = 0
filtered_func_number = 0

functions, pos = read_portion_data(FUNCTIONS_PATH, pos, 1)
while functions:
    functions, pos = read_portion_data(FUNCTIONS_PATH, pos, batch_size)
    
    for i,function in enumerate(functions):
        functions[i] = json.loads(functions[i])

    filterd_functions = filter_functions_name(functions, names)
    filtered_func_number += len(filterd_functions)
    
    append_functions_in_json(
        FILTERED_FUNCTIONS_PATH,
        filterd_functions
    )

In [29]:
print('Количество функций после фильтрации:', filtered_func_number)

Количество функций после фильтрации: 155501


### Трансформация узлов

Только для реальных сниппетов

In [2]:
# Выполнять только при обработке реальных сниппетов
FILTERED_FUNCTIONS_PATH = 'datasets/f.json'
DATASET_PATH = 'datasets/real_snippets.csv'

values = json.load(open('models/dictionary_9875.json', 'r'))
values = list(values.keys())

In [30]:
functions = load_functions_from_json(FILTERED_FUNCTIONS_PATH)

#### Извлечение названия функции из функции

Удаление из обучающей выборки названия функции и сохранение в отдельном массиве для дальнейшей валидации.

In [31]:
for func in functions:
    assert func[0]['type'] == 'FunctionDef'

In [32]:
labels = []
for i, func in enumerate(functions):
    labels.append(functions[i][0]['value'])
    del functions[i][0]

#### Трансформация

In [33]:
for i,function in enumerate(functions):
    for j,node in enumerate(function):
        try:
            functions[i][j] = node['value'] if node['value'] in values else node['type']
        except KeyError:
            functions[i][j] = node['type']

In [34]:
print('Количество функций после трансформации:', len(functions))

Количество функций после трансформации: 155501


In [35]:
assert len(labels) == len(functions)

In [36]:
print(labels[0], functions[0])

close ['arguments', 'args', 'self', 'defaults', 'body', 'If', 'AttributeLoad', 'self', 'attr', 'body', 'Expr', 'Call', 'AttributeLoad', 'AttributeLoad', 'self', 'attr', 'attr']


In [37]:
ave = sum(len(func) for func in functions) / len(functions)
print('Средняя длина вектора АСД: ' + str(ave))

Средняя длина вектора АСД: 49.860245271734584


# Сохранение финального датасета

Соединение функции с ее меткой (названием)

In [38]:
functions_dataset = [(label, func) for (label, func) in zip(labels, functions)]

Сохранение финального датасета для обучения в `.csv` файл

In [39]:
dataset_save_to_csv(DATASET_PATH, functions_dataset)

In [ ]:
#end